# Predicting stroke based on gender, age, smoking and various disease

In [2]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset
import pandas as pd
from azure.storage.filedatalake import DataLakeServiceClient
ws = Workspace.from_config()

In [3]:
import os, uuid, sys



In [100]:
download the dataset to your working directory using storage account connection string. 
to find the connection sting navigate to the Access Key and click on show key to copy key 1 connection string
from azure.storage.filedatalake import DataLakeFileClient

file = DataLakeFileClient.from_connection_string("DefaultEndpointsProtocol=https;AccountName=azuredatalakegen2store;AccountKey=XsYdPr92X/wHZTD8a1+KWzLdkDl4NxTW5+KZXJ5BPzsjmIDLjqgAtuAjJWvp+XwdQ5QDkQzeqw6cS1xGgi02bg==;EndpointSuffix=core.windows.net", 
                                                 file_system_name="datalakestore", file_path=r"file/Order Details.csv")

with open("./Order Details.csv", "wb") as my_file:
    download = file.download_file()
    download.readinto(my_file)

In [101]:
# storage_account_name = 'mystoregen1'
# storage_account_key = ''
 
# def initialize_storage_account(storage_account_name, storage_account_key):
    
#     try:  
#         global service_client
 
#         service_client = DataLakeServiceClient(account_url="adl://mystoregen1.azuredatalakestore.net/".format(
#             "https", storage_account_name), credential=storage_account_key)
    
#     except Exception as e:
#         print(e)

In [4]:
#use read the data using the dataset url from your storage location
dataset_df = pd.read_csv(r"https://azuredatalakegen2store.blob.core.windows.net/datalakestore/file2/stroke_dataset.csv")

dataset_df.head(3)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1


                          Prepare and clean the dataset

In [5]:
#check the number of rows and columns in the dataset
dataset_df.shape

(5110, 12)

In [6]:
#check for null values

dataset_df.isnull().values.any()

True

In [105]:
dataset_df.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [7]:
#have two options for BMI column which is null either we drop the column
#since it is the only one having null values of we replace the value by 0

dataset_df = dataset_df.fillna(0)


In [107]:
#check to ensure that all the null cases has been filled with 0
dataset_df.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [8]:
#list all the columns not necessary in our model
col_to_drop = ["id", "ever_married", "work_type", "Residence_type"]

for rows in col_to_drop:
    dataset_df.pop(rows)

In [9]:
dataset_df.head(3)

,gender,age,hypertension,heart_disease,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,202.21,0.0,never smoked,1
2,Male,80.0,0,1,105.92,32.5,never smoked,1


In [10]:
#the gender and Smoking_status represent categorical machine learning values 
#convert them to into discrete columns indicating variables called dummy

dataset_df = pd.get_dummies(dataset_df, columns=['gender', 'smoking_status'])
dataset_df.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,0,1,0,0,1,0,0
1,61.0,0,0,202.21,0.0,1,1,0,0,0,0,1,0
2,80.0,0,1,105.92,32.5,1,0,1,0,0,0,1,0
3,49.0,0,0,171.23,34.4,1,1,0,0,0,0,0,1
4,79.0,1,0,174.12,24.0,1,1,0,0,0,0,1,0


        BUILD YOUR MACHINE LEARNING MODELS

In [12]:
#SPLI the dataset into test and train
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X= dataset_df.drop('stroke', axis = 1) # spliting the dataset by removing the column of interest
y = dataset_df['stroke'] # dataset with just the column of interest

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=100)

In [13]:
#since we are predicting whether one had stroke or not we use one of the classification algorithn
model = RandomForestClassifier(random_state=100)
model.fit(x_train, y_train)#fit the model

RandomForestClassifier(random_state=100)

In [14]:
#predict results

y_predict = model.predict(x_test)

print(y_predict)

[0 0 0 ... 0 0 0]


In [16]:
#check the accuracy score
model.score(x_test, y_test)



0.9556425309849967

In [19]:
prediction_actual = pd.DataFrame({})
prediction_actual["Tested Values"] = y_test
prediction_actual["Predicted Values"] = y_predict
prediction_actual.tail(5)


,Tested Values,Predicted Values
3238,0,0
3716,0,0
3862,0,0
3941,0,0
4630,0,0


In [31]:
import numpy as np
y_predict[1531]

0